In [9]:
wdecay = [ 5e-5, 1e-5, 5e-6, 1e-6]
drop_rate = [ 0.3]
p = [ 0.3]
cl_rate  = [0.1, 1e-2, 1e-3]
temp = [0.1, 0.2, 0.5, 1, 10]
reg = [0.1, 1e-2, 1e-3]

In [7]:
from itertools import product

lst_runs = []
hyperparameters = [wdecay, drop_rate, p, cl_rate, temp, reg]
for params in product(*hyperparameters):
    wdecay, drop_rate, p, cl_rate, temp, reg = params
    str_base = "python main.py  --model=HGNN --dataset=ml-1m --lrate=0.001 --weight_decay={} --drop_rate={} --p={} --cl_rate={} --temp={} --reg={} --early_stopping_steps=50".format(wdecay, drop_rate, p, cl_rate, temp, reg)
    lst_runs.append(str_base)

In [8]:
with open("run_hgnn_ml1m_1.sh", "w") as f:
    for run in lst_runs:
        f.write(run + '\n')